***

# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Открываю данные

**Импорт библиотек**

In [1]:
import pandas as pd #для вывода таблицы
from sklearn.model_selection import train_test_split # для разделение оригинального датасета на выборки
from sklearn.tree import DecisionTreeClassifier # Дерево решений
from sklearn.ensemble import RandomForestClassifier #Случайный лес
from sklearn.metrics import accuracy_score # для вычисления точности предсказаний 
from sklearn.linear_model import LogisticRegression # Логистическая регрессия
from sklearn.dummy import DummyClassifier # модель для проверки на адекватность

Считаваю датасет

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head(10))
display(df.tail(10))
df.sample(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


,calls,minutes,messages,mb_used,is_ultra
3204,86.0,658.66,47.0,14153.10,0
3205,59.0,412.81,16.0,14105.03,0
3206,76.0,586.51,54.0,14345.74,0
3207,17.0,92.39,2.0,4299.25,0
3208,164.0,1016.98,71.0,17787.52,1
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


,calls,minutes,messages,mb_used,is_ultra
1447,51.0,454.46,0.0,22857.34,0
2668,73.0,474.41,65.0,7512.26,1
877,75.0,544.23,0.0,22924.79,1
1340,32.0,244.80,13.0,7213.58,0
612,65.0,456.96,0.0,41613.58,1
0,40.0,311.90,83.0,19915.42,0
2794,128.0,989.90,6.0,24999.74,0
2051,14.0,91.81,0.0,23242.01,0
294,69.0,512.46,6.0,20024.92,0
152,60.0,414.80,0.0,14945.34,0


- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.duplicated().sum()

0

На всякий случай, проверил кое-что в датасете, чтобы убедиться, что он готов к работе

Задача, данные и условия описаны. Можно приступать к работе

## Создам выборки

Из условия ясно - целевой признак тариф, уберем его из признаков и добавим как цель

In [5]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

X_train, X_test_valid, y_train, y_test_valid = train_test_split(
    features,  
    target,  
    test_size=0.4,  
    random_state=12345  
)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_test_valid,  
    y_test_valid,  
    test_size=0.5,  
    random_state=12345  
)

Теперь у нас есть разделение - 60, 20, 20 - обучающая, валидационная, тестовая

## Исследую модели

Тк у нас стоит задача определять вид тарифа - переменная качественная - задача классификации. Модели, которые попадут на исследование - дерево решений, случайный лес, логистическая регрессия

In [6]:
for depth in range(1,11):
    modelThree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    modelThree.fit(X_train, y_train)
    predictions_valid = modelThree.predict(X_valid)
    print('max_depth =', depth, ":", accuracy_score(y_valid, predictions_valid))

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7744945567651633


Лучше всего работает с глубинной 3. Теперь посмотрю на целый лес

In [8]:
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        modelForest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        modelForest.fit(X_train, y_train) 
        predictions_valid2 = modelForest.predict(X_valid) #
        result = accuracy_score(y_valid, predictions_valid2)
        if result < best_result:
            best_model = modelForest
            best_result = result
            best_est = est
            best_depth = depth
print("RMSE наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

RMSE наилучшей модели на валидационной выборке: 0.7558320373250389 Количество деревьев: 10 Максимальная глубина: 10


Осталось посмотреть на логистическую регрессию

In [9]:
modelLog = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
modelLog.fit(X_train, y_train)


predictions_valid3 = modelLog.predict(X_valid) 
accuracy_score(y_valid, predictions_valid3)


0.7107309486780715

Лучше всего справилось дерево решение с глубиной 3

## Проверка на тесте

Проверю, как поведут себя модели на тестовых выборках

In [10]:
accuracy_score(y_test, modelThree.predict(X_test))

0.7884914463452566

Даже чуть лучше, изменю глубину и посмотрю, как измениться точность

In [11]:
for depth in range(1,11):
    modelThree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    modelThree.fit(X_train, y_train)
    print('max_depth =', depth, ":", accuracy_score(y_test, modelThree.predict(X_test)))

max_depth = 1 : 0.7356143079315708
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7791601866251944
max_depth = 4 : 0.7744945567651633
max_depth = 5 : 0.7838258164852255
max_depth = 6 : 0.776049766718507
max_depth = 7 : 0.7993779160186625
max_depth = 8 : 0.7931570762052877
max_depth = 9 : 0.7807153965785381
max_depth = 10 : 0.7884914463452566


Теперь лучше всего справляется дерево с глубиной 8

In [12]:
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        modelForest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) =est и max_depth=depth
        modelForest.fit(X_train, y_train) 
        predictions_valid2 = modelForest.predict(X_test) 
        result = accuracy_score(y_test, predictions_valid2)
        if result < best_result:
            best_model = modelForest
            best_result = result
            best_est = est
            best_depth = depth
print("RMSE наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

RMSE наилучшей модели на валидационной выборке: 0.7402799377916018 Количество деревьев: 10 Максимальная глубина: 10


При использовании модели случайного леса, результат лучших гиперпараметров не поменялся

In [13]:
predictions_valid3 = modelLog.predict(X_test) 
accuracy_score(y_test, predictions_valid3)


0.6842923794712286

Тут, ситуация явно хуже

**Итог**

- При улучшении модели и проверки ее на валидационной выборке, можно было заметить, что лучше всего справляется модель дерева решений, при проверки этих моделей на тестовых выборках, это подтвердилось, но изменился гиперпараметр для нее, глубина со значения 3 изменилась на 4.

- Случайный лес показал себя одинаково на двух выборках, с гиперпараметрами: кол-во деревьев 10, их глубина 10

- Логистическая регрессия показала себя хуже всего при решении этого вопроса

## Сравню на адекватность

In [14]:
modelDUM = DummyClassifier(strategy="most_frequent")\

modelDUM.fit(X_train, y_train)

print(accuracy_score(y_test, modelDUM.predict(X_test))) 

0.6842923794712286


**Итог**

Модель дерева решений показала точность выше, чем у этой модели(Для задачи классификации лучшей константной моделью является модель, которая для любого входа предсказывает метку наибольшего класса)

**Итоговый вывод**

- Цель. Нужно построить модель для задачи классификации, которая выберет подходящий тариф
- Я рассмтаривал дерево решений, случаный лес и логистическую ресгрессию.
- Лучшей оказалась первая модель - случайный лес с глубинной 4
- Она показала лучший результат на тестовой выборки, точно почти 80
- Однако, на валидационной выборке лучше справилась та же модель, но с глубинной 3